In [ ]:
# ============================================
#  ATG Technical Assignment - Phase 1
# Fine-tuning DistilBERT using LoRA on IMDB
# ============================================

# STEP 1: Install required packages
!pip install -q transformers datasets peft accelerate bitsandbytes

# STEP 2: Import libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import torch

# STEP 3: Load the IMDB dataset
dataset = load_dataset("imdb")
print(dataset)

# STEP 4: Load tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(tokenize, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(8000))  # smaller subset for Colab
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

# STEP 5: Load base model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# STEP 6: Apply LoRA configuration (Fixed)

from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA configuration for DistilBERT
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_lin", "v_lin"]  # added this line
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()


# STEP 7: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    push_to_hub=False
)

# STEP 8: Define Trainer
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# STEP 9: Start training
trainer.train()

# STEP 10: Evaluate
metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)

# STEP 11: Save fine-tuned model
save_path = "/content/fine_tuned_distilbert_lora"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print(f" Model saved at {save_path}")

# STEP 12: (Optional) Zip and download model
!zip -r fine_tuned_distilbert_lora.zip /content/fine_tuned_distilbert_lora
from google.colab import files
files.download("fine_tuned_distilbert_lora.zip")

# STEP 13: (Optional) Push to Hugging Face Hub
 # trainer.push_to_hub("your-hf-username/fine-tuned-distilbert-lora")

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ranjana18112004 (ranjana18112004-training-placement-cell-jngec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.358500,0.335696,0.884000,0.887269
2,0.405300,0.313515,0.890500,0.892700


Evaluation metrics: {'eval_loss': 0.3135146498680115, 'eval_accuracy': 0.8905, 'eval_f1': 0.8926996570308672, 'eval_runtime': 14.9662, 'eval_samples_per_second': 133.634, 'eval_steps_per_second': 16.704, 'epoch': 2.0}
 Model saved at /content/fine_tuned_distilbert_lora
  adding: content/fine_tuned_distilbert_lora/ (stored 0%)
  adding: content/fine_tuned_distilbert_lora/special_tokens_map.json (deflated 42%)
  adding: content/fine_tuned_distilbert_lora/adapter_config.json (deflated 55%)
  adding: content/fine_tuned_distilbert_lora/README.md (deflated 66%)
  adding: content/fine_tuned_distilbert_lora/training_args.bin (deflated 53%)
  adding: content/fine_tuned_distilbert_lora/vocab.txt (deflated 53%)
  adding: content/fine_tuned_distilbert_lora/tokenizer.json (deflated 71%)
  adding: content/fine_tuned_distilbert_lora/adapter_model.safetensors (deflated 7%)
  adding: content/fine_tuned_distilbert_lora/tokenizer_config.json (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>